In [1]:
p =  12037340738208845034383383978222801137092029451270197923071397735408251586669938291587857560356890516069961904754171956588530344066457839297755929645858769

In [2]:
a =  11116464848863953015404579872882615582435721470072701808037226672236659358145861235530764279830938006566253824939581765595814774014481950202045501019370057

In [3]:
b =  7929994832424543515682621940590901949591050994041239823498965175036014605541969152503725727477980546149790133670177545414851226849642510122057033563808087

In [4]:
F = GF(p)

In [5]:
E0 = EllipticCurve([0, 0, 0, F(a), F(b)])
E0

Elliptic Curve defined by y^2 = x^3 + 11116464848863953015404579872882615582435721470072701808037226672236659358145861235530764279830938006566253824939581765595814774014481950202045501019370057*x + 7929994832424543515682621940590901949591050994041239823498965175036014605541969152503725727477980546149790133670177545414851226849642510122057033563808087 over Finite Field of size 12037340738208845034383383978222801137092029451270197923071397735408251586669938291587857560356890516069961904754171956588530344066457839297755929645858769

In [6]:
E0.cardinality()

12037340738208845034383383978222801137092029451270197923071397735408251586670085481138030088461790938201874171652771344144043268298219947026188471598838060

In [8]:
def is_elkies(ell):
    t = E0.trace_of_frobenius()
    if ell == 2:
        return False
    else:
        return (t**2 - 4 * p).jacobi(ell) == 1

In [14]:
def frob_eigenvalues(ell):
    assert is_elkies(ell)
    t = E0.trace_of_frobenius()
    X = PolynomialRing(GF(ell), "X").gen()
    v1, v2 = (X**2 - t*X + p).roots(multiplicities=False)
    return v1, v2

In [27]:
def is_torsion_order(ell, d):
    if is_elkies(ell):
        v1, v2 = frob_eigenvalues(ell)
        d1 = v1.multiplicative_order()
        d2 = v2.multiplicative_order()
        return (d1 == d and d2 > d) or (d1 > d and d2 == d)
    else:
        return False

In [ ]:
#Time to compute isogenies

In [136]:
both_sides = dict()
one_side = dict()
both_sides[0.02] = [ell for ell in prime_range(2, 380) if is_torsion_order(ell, 1)]
one_side[0.02] = [ell for ell in prime_range(381, 2000) if is_torsion_order(ell, 1) or is_torsion_order(ell, 2)]
print both_sides[0.02]
print one_side[0.02]

[3, 5, 7, 11, 13, 17, 103]
[523, 821, 947, 1723]


In [137]:
one_side[0.1] = [ell for ell in prime_range(2, 2000) if is_torsion_order(ell, 3) and 7/800 * ell > 0.1]
one_side[0.15] = [ell for ell in prime_range(2, 2000) if is_torsion_order(ell, 4) and 7/800 * ell > 0.15]
one_side[0.24] = [ell for ell in prime_range(2, 2000) if is_torsion_order(ell, 5) and 7/800 * ell > 0.24]
one_side[0.8] = [ell for ell in prime_range(2, 2000) if is_torsion_order(ell, 7) and 7/800 * ell > 0.8]
one_side[1.15] = [ell for ell in prime_range(2, 2000) if is_torsion_order(ell, 8) and 7/800 * ell > 1.15]
one_side[1.3] = [ell for ell in prime_range(2, 2000) if is_torsion_order(ell, 9) and 7/800 * ell > 1.3]
for ell in prime_range(2, 1000):
    if is_elkies(ell):
        skip = False
        for t in one_side.keys():
            if ell in one_side[t]:
                skip = True
        if is_torsion_order(ell, 1):
            skip = True
        if not skip:
            t = n(3.5/200 * ell)
            both_sides[t] = [ell]

In [138]:
print both_sides
print one_side

{7.85750000000000: [449], 0.752500000000000: [43], 10.4825000000000: [599], 16.9225000000000: [967], 10.5175000000000: [601], 6.52750000000000: [373], 12.7225000000000: [727], 9.95750000000000: [569], 15.9425000000000: [911], 0.507500000000000: [29], 11.2525000000000: [643], 8.52250000000000: [487], 0.822500000000000: [47], 3.90250000000000: [223], 9.99250000000000: [571], 17.4475000000000: [997], 16.2575000000000: [929], 13.9475000000000: [797], 5.37250000000000: [307], 2.74750000000000: [157], 3.44750000000000: [197], 10.8325000000000: [619], 5.12750000000000: [293], 4.95250000000000: [283], 7.99750000000000: [457], 6.28250000000000: [359], 9.11750000000000: [521], 1.97750000000000: [113], 5.54750000000000: [317], 4.21750000000000: [241], 14.5075000000000: [829], 0.647500000000000: [37], 16.3975000000000: [937], 15.3475000000000: [877], 15.5225000000000: [887], 12.0925000000000: [691], 12.2675000000000: [701], 0.0200000000000000: [3, 5, 7, 11, 13, 17, 103], 13.3175000000000: [761], 7

In [139]:
# Finding tje correct number of steps for each prime
def try_max_time_for_each_prime(T):
    KSS = 1 #key space size
    TT = 0.0 #total time for a walk
    steps = dict()
    #both sides
    for t in both_sides.keys():
        nsteps = (T/t).floor()
        for ell in both_sides[t]:
            steps[ell] = nsteps
            KSS *= (2*nsteps + 1)
            TT += nsteps * t
    #one side
    for t in one_side.keys():
        nsteps = (T/t).floor()
        for ell in one_side[t]:
            steps[ell] = nsteps
            KSS *= (nsteps + 1)
            TT += nsteps * t
    return KSS >= 2**256, KSS, n(KSS.log(2)), TT, steps

In [144]:
try_max_time_for_each_prime(8.2)

(True,
 316015647387021420523928003230789145321453269344234884580485478515625000000000,
 257.448459300960,
 464.955000000000,
 {3: 409,
  5: 409,
  7: 409,
  11: 409,
  13: 409,
  17: 409,
  19: 81,
  23: 20,
  29: 16,
  31: 34,
  37: 12,
  41: 11,
  43: 10,
  47: 9,
  61: 34,
  71: 6,
  73: 6,
  89: 5,
  103: 409,
  107: 4,
  109: 4,
  113: 4,
  131: 3,
  151: 3,
  157: 2,
  163: 2,
  167: 2,
  191: 2,
  193: 2,
  197: 2,
  223: 2,
  229: 2,
  241: 1,
  251: 1,
  257: 1,
  277: 1,
  283: 1,
  293: 1,
  307: 1,
  317: 1,
  349: 1,
  359: 1,
  373: 1,
  383: 1,
  401: 1,
  421: 1,
  431: 1,
  433: 1,
  439: 1,
  443: 1,
  449: 1,
  457: 1,
  467: 1,
  487: 0,
  499: 0,
  521: 0,
  523: 409,
  541: 0,
  547: 10,
  569: 0,
  571: 0,
  587: 0,
  599: 0,
  601: 0,
  607: 0,
  619: 0,
  641: 0,
  643: 0,
  659: 0,
  661: 81,
  683: 0,
  691: 0,
  701: 0,
  727: 0,
  739: 0,
  751: 0,
  761: 0,
  797: 0,
  809: 0,
  821: 409,
  829: 0,
  859: 0,
  863: 0,
  877: 0,
  881: 7,
  883: 0,
  887: 